In [1]:
import random
import time
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from tqdm import tqdm
import gc
from pyDOE import lhs
import time

# Загрузка данных
data = pd.read_csv('houston_short_300.csv')
# data = data.sample(n=300, random_state=42)
# data.to_csv('houston_short_300.csv', index=False)
X = data[['Latitude', 'Longitude', 'Year Built', 'Beds', 'Baths', 'buildingSize', 'lotSize', 'PostalCode']]
y = data['Price']

In [34]:
import random
import numpy as np
import pandas as pd
import gc
import time
from pyDOE import lhs
from sklearn.model_selection import cross_val_score
import lightgbm as lgb

# Универсальная функция кэширования с возможностью очистки кэша
class CacheFunction:
    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, x):
        # Преобразуем массив в кортеж, чтобы использовать его в качестве ключа
        key = tuple(x) if isinstance(x, (list, np.ndarray)) else x
        if key in self.cache:
            return self.cache[key]
        result = self.func(x)
        self.cache[key] = result
        return result

    def clear_cache(self):
        self.cache.clear()


# Функция оценки модели LGBM с использованием MSE
def lgbm_function(individual):
    params = {
        'n_estimators': int(individual[0]),
        'max_depth': int(individual[1]),
        'num_leaves': int(individual[2]),
        'learning_rate': individual[3],
        'max_bin': int(individual[4]),
        'colsample_bytree': individual[5]
    }

    # Принудительная проверка и коррекция параметров перед запуском модели
    if params['colsample_bytree'] > 1.0:
        params['colsample_bytree'] = 1.0
    if params['learning_rate'] > 1.0:
        params['learning_rate'] = 1.0
    if params['learning_rate'] < 0.01:
        params['learning_rate'] = 0.01
    
    # Остальные параметры в зависимости от задачи можно добавить здесь

    # print(f"Запуск кросс-валидации LGBM с параметрами: {params}")
    
    model = lgb.LGBMRegressor(**params)
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error')
    mse = -np.mean(scores)
    return mse,


# Кэшированная версия функции
cached_objective_function = CacheFunction(lgbm_function)

# Ограничение значений после мутации и кроссинговера
def check_bounds(individual, param_ranges):
    # Для каждого параметра проверяем его допустимые границы
    for i in range(len(individual)):
        lower_bound = param_ranges[i][0]
        upper_bound = param_ranges[i][1]
        # Принудительно обрезаем значения, если они выходят за границы
        individual[i] = np.clip(individual[i], lower_bound, upper_bound)
    return individual






# Инициализация популяции (латинский гиперкуб или случайная инициализация)
# Инициализация популяции (латинский гиперкуб или случайная инициализация)
def initialize_population(pop_size, param_ranges):
    population = np.random.uniform([param[0] for param in param_ranges], [param[1] for param in param_ranges], (pop_size, len(param_ranges)))
    
    # Применяем check_bounds ко всей начальной популяции и преобразуем в массив NumPy
    population = np.array([check_bounds(ind, param_ranges) for ind in population])
    # print(f"Проверка популяции после инициализации: {population}")
    
    return population



def select_top_individuals_with_elitism(population, fitness_values, selection_percentage, elite_count=1):
    num_selected = int(len(population) * selection_percentage) - elite_count
    sorted_indices = np.argsort(fitness_values)

    # Преобразуем формы массивов с помощью np.squeeze(), чтобы избавиться от лишних осей
    elite_indices = np.squeeze(sorted_indices[:elite_count])
    elite_population = np.squeeze(population[elite_indices])
    elite_fitness = np.squeeze(fitness_values[elite_indices])

    selected_indices = np.squeeze(sorted_indices[elite_count:elite_count + num_selected])
    selected_population = np.squeeze(population[selected_indices])
    selected_fitness = np.squeeze(fitness_values[selected_indices])

    final_population = np.concatenate([elite_population, selected_population])
    final_fitness = np.concatenate([elite_fitness, selected_fitness])

    return final_population, final_fitness





# Скрещивание двух родителей
def crossover(parent1, parent2):
    return np.random.uniform(parent1, parent2)

# Создание потомков
def generate_offspring(selected_population, num_offspring):
    offspring = []
    for _ in range(num_offspring):
        parents = np.random.choice(len(selected_population), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        offspring.append(child)
    return np.array(offspring)

# Мутация потомков
def mutate(offspring, mutation_rate, mutation_strength, mutation_percent_genes, mutation_by_replacement, param_ranges):
    num_to_mutate = int(len(offspring) * mutation_rate)
    indices_to_mutate = np.random.choice(len(offspring), num_to_mutate, replace=False)
    for i in indices_to_mutate:
        num_genes_to_mutate = int(offspring.shape[1] * mutation_percent_genes)
        genes_to_mutate = np.random.choice(offspring.shape[1], num_genes_to_mutate, replace=False)
        for gene_idx in genes_to_mutate:
            if mutation_by_replacement:
                offspring[i, gene_idx] = np.random.uniform(param_ranges[gene_idx][0], param_ranges[gene_idx][1])
            else:
                mutation = np.random.uniform(-mutation_strength, mutation_strength) * (param_ranges[gene_idx][1] - param_ranges[gene_idx][0])
                offspring[i, gene_idx] += mutation
                # Применяем функцию check_bounds сразу после мутации
                offspring[i, gene_idx] = np.clip(offspring[i, gene_idx], param_ranges[gene_idx][0], param_ranges[gene_idx][1])

    return offspring




# Оптимизация генетическим алгоритмом genalg
class GeneticAlgorithmOptimizer:
    def __init__(self, param_ranges, population_size, offspring_ratio, mutation_rate, mutation_strength, mutation_percent_genes,
                 mutation_by_replacement, elite_fraction, initialization_method, selection_percentage, early_stopping_patience, generations=2):
        self.param_ranges = param_ranges
        self.population_size = population_size
        self.offspring_ratio = offspring_ratio
        self.mutation_rate = mutation_rate
        self.mutation_strength = mutation_strength
        self.mutation_percent_genes = mutation_percent_genes
        self.mutation_by_replacement = mutation_by_replacement
        self.elite_fraction = elite_fraction
        self.initialization_method = initialization_method
        self.selection_percentage = selection_percentage
        self.early_stopping_patience = early_stopping_patience
        self.generations = generations
    
    def optimize(self):
        # Инициализация популяции
        population = initialize_population(self.population_size, self.param_ranges)
        # print(f"Инициализация популяции: {np.array(population).shape}")
        
        # Оценка начальной популяции
        fitness_values = np.array([cached_objective_function(tuple(ind)) for ind in population])
        fitness_values = np.squeeze(fitness_values) 
        # print(f"Оценка начальной популяции, первая особь: {fitness_values[:1]}")
        
        best_solution = None
        best_fitness = float('inf')
        no_improvement_count = 0

        global_start_time = time.time()

        for generation in range(self.generations):
            print(f"\nПоколение {generation + 1}")
            
            # Селекция лучших индивидов с элитизмом
            selected_population, selected_fitness = select_top_individuals_with_elitism(
            np.squeeze(population), np.squeeze(fitness_values), 
            self.selection_percentage, elite_count=int(self.elite_fraction * self.population_size)
            )
            
            # Контроль индексов и значений
            # print(f"Индексы элитных особей: {selected_population[:10]}")
            # print(f"Размер выбранной популяции: {np.array(selected_population).shape}, лучший фитнес: {selected_fitness[:1]}")
            
            # Создание потомков через скрещивание
            num_offspring = int(len(selected_population) * self.offspring_ratio)
            offspring = generate_offspring(selected_population, num_offspring)
            # print(f"Создано потомков: {np.array(offspring).shape}")
            
            # Применение мутации к потомкам
            offspring = mutate(offspring, self.mutation_rate, self.mutation_strength, self.mutation_percent_genes,
                               self.mutation_by_replacement, self.param_ranges)
            
            # Применение функции check_bounds ко всем потомкам
            offspring = [check_bounds(ind, self.param_ranges) for ind in offspring]
            # print(f"Проверка потомков после мутации: {offspring}")

            # Применяем check_bounds к selected_population
            selected_population = np.array([check_bounds(ind, self.param_ranges) for ind in selected_population])


            # Объединение потомков и выбранных индивидов
            combined_population = np.concatenate([selected_population, offspring])
            combined_population = np.squeeze(combined_population)

            # Применяем check_bounds ко всей популяции
            combined_population = np.array([check_bounds(ind, self.param_ranges) for ind in combined_population])

            # Проверка каждого индивида, что он в пределах допустимых границ
            # for ind in combined_population:
            #     if not np.all(np.logical_and(ind >= [p[0] for p in self.param_ranges], ind <= [p[1] for p in self.param_ranges])):
            #         print(f"Индивид вне допустимых границ: {ind}")
            #     else:
            #         print(f"Индивид в пределах: {ind}")

            
            # Оценка новой популяции с преобразованием numpy.ndarray в tuple
            combined_fitness = np.array([cached_objective_function(tuple(ind.flatten())) for ind in combined_population])
            combined_fitness = np.squeeze(combined_fitness)
            # print(f"Лучший фитнес в новом поколении: {min(combined_fitness)}")

            # Селекция лучших индивидов для следующего поколения
            final_population, final_fitness = select_top_individuals_with_elitism(
                combined_population, combined_fitness, self.population_size / len(combined_population),
                elite_count=int(self.elite_fraction * self.population_size)
            )

            population = final_population
            fitness_values = final_fitness

            # Определение лучшего решения
            current_best_fitness = np.min(final_fitness)

            current_best_solution = np.squeeze(final_population[np.argmin(final_fitness)])


            if current_best_fitness < best_fitness:
                best_fitness = current_best_fitness
                best_solution = current_best_solution
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            # Выводим лучший результат и параметры для текущего поколения
            print(f"Поколение {generation + 1}: Лучший MSE: {best_fitness}")

            
            # Раннее завершение, если нет улучшений
            if no_improvement_count >= self.early_stopping_patience:
                break

        elapsed_time = time.time() - global_start_time
        return best_solution, best_fitness, generation + 1, elapsed_time






# Генерация случайных гиперпараметров
def generate_random_hyperparameters():
    hyperparameters = {
        "population_size": random.randint(10, 50),
        "offspring_ratio": random.uniform(1.5, 3.0),
        "mutation_rate": random.uniform(0.1, 0.5),
        "mutation_strength": random.uniform(0.1, 0.5),
        "mutation_percent_genes": random.uniform(0.1, 0.2),
        "mutation_by_replacement": random.choice([True]),
        "elite_fraction": random.uniform(0.05, 0.2),
        "initialization_method": random.choice(['random', 'lhs'])
    }
    return hyperparameters

# Количество внешних и внутренних прогонов
num_outer_runs = 20  # Число внешних циклов для тестирования разных гиперпараметров
num_inner_runs = 5   # Число внутренних циклов для усреднения результатов одного набора гиперпараметров
summary_results = []  # Список для хранения итоговых результатов
csv_filename = 'cust_houses.csv'  # Имя файла для сохранения результатов

# Внешний цикл для тестирования разных гиперпараметров
for outer_run in range(num_outer_runs):
    print(f"\n=== Внешний прогон {outer_run + 1} ===\n")
    
    # Генерация случайных гиперпараметров
    hyperparams = generate_random_hyperparameters()
    # print(f"Сгенерированные гиперпараметры для внешнего прогона {outer_run + 1}: {hyperparams}")
    
    results = []  # Список для хранения результатов одного внешнего цикла

    # Внутренний цикл для усреднения результатов одного набора гиперпараметров
    for inner_run in range(num_inner_runs):
        # Инициализация оптимизатора генетического алгоритма
        optimizer = GeneticAlgorithmOptimizer(
            param_ranges=[
                (50, 2000),    # n_estimators
                (1, 100),      # max_depth
                (2, 1000),     # num_leaves
                [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],   # learning_rate
                (10, 1000),    # max_bin
                [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]     # colsample_bytree
            ],
            **hyperparams,  # Передаем гиперпараметры
            generations=30,  # Количество поколений для генетического алгоритма
            selection_percentage=0.5,
            early_stopping_patience=10
        )

        # Оптимизация для данного набора гиперпараметров
        best_solution, best_fitness, best_generation, best_generation_time = optimizer.optimize()

        # Сохранение результатов внутреннего прогона
        run_data = {
            'Best MSE': best_fitness,
            'Best Generation': best_generation,
            'Time to Best Generation': best_generation_time,
            'Population Size': hyperparams["population_size"],
            'Mutation Rate': hyperparams["mutation_rate"],
            'Offspring Ratio': hyperparams["offspring_ratio"],  # Сохраняем правильно!
            'Mutation Strength': hyperparams["mutation_strength"],
            'Mutation Percent Genes': hyperparams["mutation_percent_genes"],
            'Elite Fraction': hyperparams["elite_fraction"]
        }
        results.append(run_data)

    # Обработка результатов внутреннего цикла
    df_results = pd.DataFrame(results)
    median_fitness = df_results['Best MSE'].median()
    mode_iteration = df_results['Best Generation'].mode()[0]
    median_time_to_best_gen = df_results['Time to Best Generation'].median()

    # Подготовка строки для сохранения в CSV
    summary_row = {
        'Median MSE': median_fitness,
        'Mode Best Generation': mode_iteration,
        'Median Time to Best Generation': median_time_to_best_gen,
        'Population Size': hyperparams["population_size"],
        'Mutation Rate': hyperparams["mutation_rate"],
        'Offspring Ratio': hyperparams["offspring_ratio"],
        'Mutation Strength': hyperparams["mutation_strength"],
        'Mutation Percent Genes': hyperparams["mutation_percent_genes"],
        'Elite Fraction': hyperparams["elite_fraction"]
    }

    summary_results.append(summary_row)

    # Добавляем строку в CSV-файл после каждого внешнего прогона
    df_summary = pd.DataFrame([summary_row])
    df_summary.to_csv(csv_filename, mode='a', header=not pd.io.common.file_exists(csv_filename), index=False)

    # Очистка памяти после каждого внешнего цикла
    gc.collect()

# Вывод итоговых результатов после всех внешних прогонов
df_summary = pd.DataFrame(summary_results)
print("\nИтоговая таблица с медианными значениями и гиперпараметрами:")
print(df_summary)




=== Внешний прогон 1 ===


Поколение 1
Поколение 1: Лучший MSE: 514552.534131664

Поколение 2
Поколение 2: Лучший MSE: 510606.49938187044

Поколение 3
Поколение 3: Лучший MSE: 510606.49938187044

Поколение 4
Поколение 4: Лучший MSE: 510606.49938187044

Поколение 5
Поколение 5: Лучший MSE: 510606.49938187044

Поколение 6
Поколение 6: Лучший MSE: 510606.49938187044

Поколение 7
Поколение 7: Лучший MSE: 510606.49938187044

Поколение 8
Поколение 8: Лучший MSE: 510606.49938187044

Поколение 9
Поколение 9: Лучший MSE: 510606.49938187044

Поколение 10
Поколение 10: Лучший MSE: 510538.29856219963

Поколение 11
Поколение 11: Лучший MSE: 510538.29856219963

Поколение 12
Поколение 12: Лучший MSE: 510538.29856219963

Поколение 13
Поколение 13: Лучший MSE: 510489.3503243143

Поколение 14
Поколение 14: Лучший MSE: 510482.02652173024

Поколение 15
Поколение 15: Лучший MSE: 510482.02652173024

Поколение 16
Поколение 16: Лучший MSE: 510482.0256733026

Поколение 17
Поколение 17: Лучший MSE: 510482.0256